- Nilai Numerik -> imputasi dengan modus
- Nilai kategori -> imputasi dengan Ffill
- Outlier dengan -> rentang z score

In [212]:
import pandas as pd


In [213]:
df = pd.read_csv('C:\\Users\\thebl\\Downloads\\Clone\\Intern-BCC-2024\\Dataset\\Raw\\climate_vs_flood.csv')

In [214]:
df.isnull().sum()

date               0
Tn               312
Tx               213
Tavg              46
RH_avg            52
RR              2315
ss              1259
ff_x              22
ddd_x             22
ff_avg            93
ddd_car          101
station_id         0
station_name       0
region_name        0
flood              0
dtype: int64

Imputasi nilai numerikal dengan modus

In [216]:
fitur_fitur = ['Tx', 'Tn', 'Tavg', 'RH_avg', 'ss','RR', 'ff_x', 'ff_avg', 'ddd_x']
for column in fitur_fitur:
    mode_value = df[column].mode()[0]
    df[column] = df[column].fillna(mode_value)

In [217]:
df.isnull().sum()


date              0
Tn                0
Tx                0
Tavg              0
RH_avg            0
RR                0
ss                0
ff_x              0
ddd_x             0
ff_avg            0
ddd_car         101
station_id        0
station_name      0
region_name       0
flood             0
dtype: int64

Imputasi nilai kategori dengan Ffill

In [218]:
df['ddd_car'] = df['ddd_car'].bfill()

In [219]:
df.isnull().sum()

date            0
Tn              0
Tx              0
Tavg            0
RH_avg          0
RR              0
ss              0
ff_x            0
ddd_x           0
ff_avg          0
ddd_car         0
station_id      0
station_name    0
region_name     0
flood           0
dtype: int64

In [220]:
num_col = ['Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg']
cat_col = ['ddd_car','region_name']
target = ['flood']

In [221]:
df_train = pd.concat([df[num_col], df[cat_col]], axis=1)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6308 entries, 0 to 6307
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Tn           6308 non-null   float64
 1   Tx           6308 non-null   float64
 2   Tavg         6308 non-null   float64
 3   RH_avg       6308 non-null   float64
 4   RR           6308 non-null   float64
 5   ss           6308 non-null   float64
 6   ff_x         6308 non-null   float64
 7   ddd_x        6308 non-null   float64
 8   ff_avg       6308 non-null   float64
 9   ddd_car      6308 non-null   object 
 10  region_name  6308 non-null   object 
dtypes: float64(9), object(2)
memory usage: 542.2+ KB


In [222]:
df_train_dummied = pd.get_dummies(df_train, columns=['ddd_car', 'region_name'], drop_first=True)

In [223]:
from imblearn.over_sampling import RandomOverSampler    
X = df_train_dummied
y = df[target]

# Inisialisasi RandomOverSampler
oversampler = RandomOverSampler(random_state=42)

# Melakukan oversampling
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Menampilkan jumlah sampel setelah oversampling
print("Jumlah sampel setelah oversampling:")
print(y_resampled.value_counts())

Jumlah sampel setelah oversampling:
flood
0        5832
1        5832
Name: count, dtype: int64


In [224]:
from sklearn.model_selection import train_test_split

# Pembagian dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print("Jumlah data latih:", len(X_train))
print("Jumlah data uji:", len(X_test))


Jumlah data latih: 9331
Jumlah data uji: 2333


In [225]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150, max_depth=15, random_state=42)
rf.fit(X_train, y_train)

c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=15, n_estimators=150, random_state=42)

In [226]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.947278182597514


In [227]:
from sklearn.metrics import classification_report

# Evaluasi performa model
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.92      0.95      1205
           1       0.92      0.97      0.95      1128

    accuracy                           0.95      2333
   macro avg       0.95      0.95      0.95      2333
weighted avg       0.95      0.95      0.95      2333



In [228]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(rf, X, y, cv=5)

# Menampilkan skor validasi silang untuk setiap fold
print("Cross-validation Scores:", cv_scores)

# Menampilkan rata-rata skor validasi silang
print("Mean CV Score:", cv_scores.mean())

c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConvers

Cross-validation Scores: [0.79793978 0.9207607  0.92393027 0.92783505 0.27835052]
Mean CV Score: 0.7697632623719508


In [229]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# find best scored 5 features
select_feature = SelectKBest(chi2, k=5).fit(X_train, y_train)

In [230]:
print('Score list:', select_feature.scores_)
print('Feature list:', X_train.columns)

Score list: [4.61412794e+01 5.36951576e+01 9.85036598e+01 1.57847358e+03
 4.09842977e+04 1.35203471e+03 1.76670850e+01 2.81762984e+03
 4.19387363e+00 5.85708886e-01 5.73241866e+00 1.12307149e+01
 1.09523041e+02 2.35062437e+02 4.59384428e+01 3.47325618e+02
 3.73981481e+02 6.57727969e+02 6.45125610e+02 1.33612469e+00]
Feature list: Index(['Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg',
       'ddd_car_E ', 'ddd_car_N ', 'ddd_car_NE', 'ddd_car_NW', 'ddd_car_S ',
       'ddd_car_SE', 'ddd_car_SW', 'ddd_car_W ', 'region_name_Jakarta Selatan',
       'region_name_Jakarta Timur', 'region_name_Jakarta Utara'],
      dtype='object')
